### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import tensorflow as tf

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_data_header, males_keys, females_keys

H5_dic = read_data_header()
chunk_size = 512*30

males = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'


Data with 40 male and female speakers


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model pretrained loading

In [4]:
learning_rate = 0.0001
batch_size = 1
N = 256
max_pool = 256
regularization = 0.0001

config_model = {}
config_model["alpha"] = learning_rate
config_model["reg"] = regularization
config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size
config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["type"] = "DPCL_train_front"
config_model["smooth_size"] = 10
idd = ''.join('-{}={}-'.format(key, val) for key, val in sorted(config_model.items()))
config_model["type"] = "DPCL_fine_tuning"

In [5]:
from models.adapt import Adapt

full_id = 'shiny-frog-0617'+idd

folder='DPCL_fine_tuning'
model = Adapt(config_model=config_model,pretraining= False, folder=folder)

from models.dpcl import DPCL

with model.graph.as_default():
    model.connect_front(DPCL)
    model.sepNet.output = model.sepNet.prediction
    model.create_saver()
    model.restore_model('DPCL_train_front', full_id)
    model.sepNet.output = model.sepNet.separate
    model.separator
    model.back
    model.cost
    model.optimize
    model.tensorboard_init()



ID : AdaptiveNet-fragrant-lake-4285
INFO:tensorflow:Restoring parameters from /home/anthony/das/log/DPCL_train_front/AdaptiveNet-shiny-frog-0617-N=256--alpha=0.0001--batch_size=1--chunk_size=15360--maxpool=256--reg=0.0001--smooth_size=10--type=DPCL_train_front-/model.ckpt


ValueError: Tensor("separate/GatherNd:0", shape=(?, 2, 40), dtype=float32) must be from the same graph as Tensor("network/Const:0", shape=(), dtype=int32).

## Connect DPCL model to the front end and back end

In [ ]:
with model.graph.as_default():
    uninit =[v for v in tf.global_variables() if v.name.split(':')[0] in set(model.sess.run(tf.report_uninitialized_variables()))]
    print uninit
    init = tf.variables_initializer(uninit)

# Model creation

# Pretraining the model 

In [ ]:
nb_iterations = 1000

#initialize the model
model.sess.run(init)

for i in range(nb_iterations):
    X_in, X_mix, Ind = mixed_data.get_batch(batch_size)
    c = model.train(X_mix, X_in, learning_rate, i)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        model.save(i)